In [26]:
!nvidia-smi

Sun Nov  8 18:14:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   36C    P0    40W / 300W |     10MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [21]:
import sys
print(sys.version)

3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) 
[GCC 7.5.0]


In [22]:
import pylab
import imageio
import cv2
import os
from glob import glob
from tqdm import tqdm,tnrange
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import tensorflow as tf

In [23]:
# """
# #GPU 메모리 제한하기
# https://www.tensorflow.org/guide/gpu#allowing_gpu_memory_growth
# 수정 필요
# """
# tf.debugging.set_log_device_placement(True)

# # 텐서 생성
# a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
# b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
# c = tf.matmul(a, b)

# print(c)

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         tf.config.experimental.set_memory_growth(gpus[0], True)
#     except RuntimeError as e:
#         # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
#         print(e)

## Model pipeline config

1. class 갯수 수정
2. fine_tune_checkpoint 경로 수정
3. fine_tune_checkpoint_type 수정
4. train_input_reader 의 label_map_path(.pbtxt), input_path(tfrecord) 경로 수정
5. 배치사이즈, 학습률 조정
6. max_number_of_boxes 조정(ValueError("Shapes %s and %s are incompatible" % (self, other)) 에러로 인한 갯수 대폭 증가)

In [24]:
#protocol buffer compile
!protoc /home/workspace/user-workspace/object_tracking/madfalcon/tf_models/research/object_detection/protos/*.proto --python_out=.

/home/workspace/user-workspace/object_tracking/madfalcon/tf_models/research/object_detection/protos/anchor_generator.proto: File does not reside within any path specified using --proto_path (or -I).  You must specify a --proto_path which encompasses this file.  Note that the proto_path must be an exact prefix of the .proto file names -- protoc is too dumb to figure out when two paths (e.g. absolute and relative) are equivalent (it's harder than you think).


## Efndet

In [28]:
"""
1. 참고 :
    PIPELINE_CONFIG_PATH : 모델 파이프라인 config 파일경로 지정
    MODEL_DIR : 학습 후 저장될 모델경로 지정
    1. model config 관련:
        https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_training_and_evaluation.md
        
    2. 학습 중 loss 값이 nan으로 나오는경우 :
        https://github.com/tensorflow/models/issues/9151
        결론: 배치사이즈를 줄이게 되면 학습률도 변경해봐라 ex) batch size = 6 일때 LR = 0.003 
        
    3. 텐서보드로 학습결과 보는 법:
        https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#monitor-training-job-progress-using-tensorboard

"""
for i in range(1):
    PIPELINE_CONFIG_PATH=f'/home/workspace/user-workspace/object_tracking/madfalcon/efficientdet_d1_coco17_tpu-32/pipeline_cam_{str(i)}.config'
    MODEL_DIR=f'/home/workspace/user-workspace/object_tracking/madfalcon/result_{str(i)}/'
    !python '/home/workspace/user-workspace/object_tracking/madfalcon/tf_models/research/20201104_model_main_tf2.py' \
        --pipeline_config_path=$PIPELINE_CONFIG_PATH \
        --model_dir=$MODEL_DIR \
        --alsologtostderr

2020-11-08 18:14:21.176779: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-08 18:14:23.646009: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-08 18:14:23.650213: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-08 18:14:23.651465: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:05.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.72GiB deviceMemoryBandwidth: 836.37GiB/s
2020-11-08 18:14:23.651527: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-08 18:14:23.653835: I tensorflow/stream_executor/plat

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I1108 18:14:25.646178 140661134821184 cross_device_ops.py:441] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I1108 18:14:25.648961 140661134821184 cross_device_ops.py:441] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I1108 18:14:25.659472 140661134821184 efficientnet_model.py:148] round_filter input=32 output=32
I1108 18:14:25.659598 140661134821184 efficientnet_model.py:148] round_filter input=16 output=16
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I1108 18:14:

Instructions for updating:
Use `tf.cast` instead.
W1108 18:14:41.901134 140661134821184 deprecation.py:317] From /home/workspace/user-workspace/object_tracking/madfalcon/tf_models/research/object_detection/inputs.py:281: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
W1108 18:14:48.313595 140621876532992 deprecation.py:317] From /home/workspace/user-workspace/object_tracking/madfalcon/tf_models/research/object_detection/model_lib_v2.py:355: set_learning_phase (from tensorflow.python.keras.backend) is deprecated and will be removed after 2020-10-11.
Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
2020-11-08 18:15:17.662605: I tensorflow/stream_executor/platform/defaul

W1108 18:17:11.264668 140621884925696 optimizer_v2.py:1273] Gradients do not exist for variables ['stack_6/block_1/expand_bn/gamma:0', 'stack_6/block_1/expand_bn/beta:0', 'stack_6/block_1/depthwise_conv2d/depthwise_kernel:0', 'stack_6/block_1/depthwise_bn/gamma:0', 'stack_6/block_1/depthwise_bn/beta:0', 'stack_6/block_1/project_bn/gamma:0', 'stack_6/block_1/project_bn/beta:0', 'top_bn/gamma:0', 'top_bn/beta:0'] when minimizing the loss.
Instructions for updating:
Use fn_output_signature instead
W1108 18:17:12.787236 140621884925696 deprecation.py:500] From /home/workspace/user-workspace/lib/tensorflow/python/util/deprecation.py:574: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Use fn_output_signature instead
W1108 18:17:25.219881 140621884925696 optimizer_v2.py:1273] Gradients do not exist for variables ['stack_6/block_1/expand_bn/gamma:0', 'stack_6/block_1/expand_bn/beta:0', 'stack_6/

INFO:tensorflow:Step 3900 per-step time 0.767s loss=2.900
I1108 19:11:54.363686 140661134821184 model_lib_v2.py:649] Step 3900 per-step time 0.767s loss=2.900
INFO:tensorflow:Step 4000 per-step time 0.756s loss=2.271
I1108 19:13:12.182510 140661134821184 model_lib_v2.py:649] Step 4000 per-step time 0.756s loss=2.271
INFO:tensorflow:Step 4100 per-step time 0.781s loss=2.847
I1108 19:14:30.935300 140661134821184 model_lib_v2.py:649] Step 4100 per-step time 0.781s loss=2.847
INFO:tensorflow:Step 4200 per-step time 0.785s loss=2.394
I1108 19:15:48.979842 140661134821184 model_lib_v2.py:649] Step 4200 per-step time 0.785s loss=2.394
INFO:tensorflow:Step 4300 per-step time 0.786s loss=3.010
I1108 19:17:07.942312 140661134821184 model_lib_v2.py:649] Step 4300 per-step time 0.786s loss=3.010
INFO:tensorflow:Step 4400 per-step time 0.790s loss=2.296
I1108 19:18:25.914037 140661134821184 model_lib_v2.py:649] Step 4400 per-step time 0.790s loss=2.296
INFO:tensorflow:Step 4500 per-step time 0.763s

INFO:tensorflow:Step 9100 per-step time 0.759s loss=3.457
I1108 20:19:37.400147 140661134821184 model_lib_v2.py:649] Step 9100 per-step time 0.759s loss=3.457
INFO:tensorflow:Step 9200 per-step time 0.765s loss=2.742
I1108 20:20:55.510991 140661134821184 model_lib_v2.py:649] Step 9200 per-step time 0.765s loss=2.742
INFO:tensorflow:Step 9300 per-step time 0.777s loss=2.791
I1108 20:22:13.467325 140661134821184 model_lib_v2.py:649] Step 9300 per-step time 0.777s loss=2.791
INFO:tensorflow:Step 9400 per-step time 0.755s loss=2.480
I1108 20:23:31.815556 140661134821184 model_lib_v2.py:649] Step 9400 per-step time 0.755s loss=2.480
INFO:tensorflow:Step 9500 per-step time 0.768s loss=2.253
I1108 20:24:49.634145 140661134821184 model_lib_v2.py:649] Step 9500 per-step time 0.768s loss=2.253
INFO:tensorflow:Step 9600 per-step time 0.785s loss=2.758
I1108 20:26:07.482503 140661134821184 model_lib_v2.py:649] Step 9600 per-step time 0.785s loss=2.758
INFO:tensorflow:Step 9700 per-step time 0.775s

## Training Model Export

In [29]:
for i in range(1):
    PIPELINE_CONFIG_PATH=f'/home/workspace/user-workspace/object_tracking/madfalcon/efficientdet_d1_coco17_tpu-32/pipeline_cam_{str(i)}.config'
    MODEL_DIR=f'/home/workspace/user-workspace/object_tracking/madfalcon/result_{str(i)}/'
    !python '/home/workspace/user-workspace/object_tracking/madfalcon/tf_models/research/20201107_exporter_main_v2.py' \
        --pipeline_config_path $PIPELINE_CONFIG_PATH  \
        --trained_checkpoint_dir $MODEL_DIR  \
        --output_directory '/home/workspace/user-workspace/object_tracking/madfalcon/result_0_export/'

2020-11-08 20:31:25.541261: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-08 20:31:27.532257: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-08 20:31:27.536073: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-08 20:31:27.537196: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:05.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.72GiB deviceMemoryBandwidth: 836.37GiB/s
2020-11-08 20:31:27.537236: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-08 20:31:27.539372: I tensorflow/stream_executor/plat

I1108 20:31:30.785457 139930250495808 efficientnet_model.py:148] round_filter input=80 output=80
I1108 20:31:30.785640 139930250495808 efficientnet_model.py:148] round_filter input=112 output=112
I1108 20:31:31.194356 139930250495808 efficientnet_model.py:148] round_filter input=112 output=112
I1108 20:31:31.194558 139930250495808 efficientnet_model.py:148] round_filter input=192 output=192
I1108 20:31:31.736924 139930250495808 efficientnet_model.py:148] round_filter input=192 output=192
I1108 20:31:31.737116 139930250495808 efficientnet_model.py:148] round_filter input=320 output=320
I1108 20:31:31.939285 139930250495808 efficientnet_model.py:148] round_filter input=1280 output=1280
I1108 20:31:31.977796 139930250495808 efficientnet_model.py:460] Building model efficientnet with params ModelConfig(width_coefficient=1.0, depth_coefficient=1.1, resolution=240, dropout_rate=0.2, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=

INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3b324f0ee0>, TensorSpec(shape=(None, 80, 80, 40), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3b324f0b80>, TensorSpec(shape=(None, 40, 40, 112), dtype=tf.float32, name='feature_pyramid/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3b324f0a90>, TensorSpec(shape=(None, 20, 20, 320), dtype=tf.float32, name='feature_pyramid/2/1'))], False), {}).
I1108 20:33:14.596579 139930250495808 def_function.py:1038] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3b324f0ee0>, TensorSpec(shape=(None, 80, 80, 40), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3b324f0b80>, TensorSpec(shape=(None, 40, 40, 112), dtype=tf.float32, name

INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3b324f0ee0>, TensorSpec(shape=(None, 80, 80, 40), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3b324f0b80>, TensorSpec(shape=(None, 40, 40, 112), dtype=tf.float32, name='feature_pyramid/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3b324f0a90>, TensorSpec(shape=(None, 20, 20, 320), dtype=tf.float32, name='feature_pyramid/2/1'))], False), {}).
I1108 20:33:26.584804 139930250495808 def_function.py:1038] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3b324f0ee0>, TensorSpec(shape=(None, 80, 80, 40), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3b324f0b80>, TensorSpec(shape=(None, 40, 40, 112), dtype=tf.float32, name

## centernet resnet50

In [ ]:
#cam 0번까지 돌려보자
for i in range(1):
    PIPELINE_CONFIG_PATH=f'/home/workspace/user-workspace/object_tracking/madfalcon/centernet_resnet50_v2_512x512_coco17_tpu-8/pipeline_cam_{i}.config'
    MODEL_DIR=f'/home/workspace/user-workspace/object_tracking/madfalcon/result_{i}_centernet/'
    !python '/home/workspace/user-workspace/object_tracking/madfalcon/tf_models/research/20201104_model_main_tf2.py' \
        --pipeline_config_path=$PIPELINE_CONFIG_PATH \
        --model_dir=$MODEL_DIR \
        --alsologtostderr